We're going to be using a recurrent neural network to model binary addition. 
Binary addition moves from right to left, where we try to predict the number beneath the line given the numbers above the line. We want the neural network to move along the binary sequences and remember when it has carried the 1 and when it hasn't, so that it can make the correct prediction. 

In [1]:
import copy, numpy as np
np.random.seed(0)

In [2]:
# compute sigmoid nonlinearity
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output

In [3]:
# convert output of sigmoid function to its derivative
def sigmoid_output_to_derivative(output):
    return output*(1-output)

In [4]:
# training dataset generation
int2binary = {}
binary_dim = 8

In [5]:
largest_number = pow(2,binary_dim)
binary = np.unpackbits(
    np.array([range(largest_number)],dtype=np.uint8).T,axis=1)
for i in range(largest_number):
    int2binary[i] = binary[i]

In [26]:
# input variables
alpha = 0.1 ##learning rate
##We are adding two numbers together, so we'll be feeding in two-bit strings one character at the time each. 
##Thus, we need to have two inputs to the network (one for each of the numbers being added).
input_dim = 2 
##This is the size of the hidden layer that will be storing our carry bit
hidden_dim = 16
output_dim = 1

In [51]:
# initialize neural network weights

##This is the matrix of weights that connects our input layer and our hidden layer. 
##Thus, it has "input_dim" rows and "hidden_dim" columns. (2 x 16)
synapse_0 = 2*np.random.random((input_dim,hidden_dim)) - 1
##This is the matrix of weights that connects the hidden layer to the output layer. 
##Thus, it has "hidden_dim" rows and "output_dim" columns. (16 x 1 unless you change it).
synapse_1 = 2*np.random.random((hidden_dim,output_dim)) - 1
##This is the matrix of weights that connects the hidden layer in the previous time-step to the hidden layer
##in the current timestep. It also connects the hidden layer in the current timestep to the hidden layer in 
##the next timestep . Thus, it has the dimensionality of "hidden_dim" rows and "hidden_dim" columns. (16 x 16).
synapse_h = 2*np.random.random((hidden_dim,hidden_dim)) - 1

##These store the weight updates that we would like to make for each of the weight matrices. 
##After we've accumulated several weight updates, we'll actually update the matrices
synapse_0_update = np.zeros_like(synapse_0)
synapse_1_update = np.zeros_like(synapse_1)
synapse_h_update = np.zeros_like(synapse_h)

In [52]:
# training logic
for j in range(10000):
    
    # generate a simple addition problem (a + b = c)
    a_int = np.random.randint(largest_number/2) # int version
    a = int2binary[a_int] # binary encoding

    b_int = np.random.randint(largest_number/2) # int version
    b = int2binary[b_int] # binary encoding

    # true answer
    c_int = a_int + b_int
    c = int2binary[c_int]
    
    # where we'll store our best guess (binary encoded)
    d = np.zeros_like(c)

    overallError = 0
    
    layer_2_deltas = list() ## layer2 derivatives
    layer_1_values = list() ## layer1 value
    layer_1_values.append(np.zeros(hidden_dim)) ## initialize layer1
    
    # moving along the positions in the binary encoding
    for position in range(binary_dim):
        
        # generate input and output
        ##X is the same as "layer_0". X is a list of 2 numbers, one from a and one from b. 
        ##It's indexed according to the "position" variable, but we index it in such a way that it goes from 
        ##right to left. So, when position == 0, this is the farhest bit to the right in "a" and the farthest 
        ##bit to the right in "b". When position equals 1, this shifts to the left one bit.
        X = np.array([[a[binary_dim - position - 1],b[binary_dim - position - 1]]])
        y = np.array([[c[binary_dim - position - 1]]]).T

        # hidden layer (input ~+ prev_hidden)
        ## here magic happens!! the prev hidden layers are dotted with weight matrix at layer 2 and summed
        ## with the dot product of input and weight matrix at layer 1
        layer_1 = sigmoid(np.dot(X,synapse_0) + np.dot(layer_1_values[-1],synapse_h))

        # output layer (new binary representation)
        layer_2 = sigmoid(np.dot(layer_1,synapse_1))

        # did we miss?... if so, by how much?
        layer_2_error = y - layer_2
        layer_2_deltas.append((layer_2_error)*sigmoid_output_to_derivative(layer_2))
        overallError += np.abs(layer_2_error)
    
        # decode estimate so we can print it out
        d[binary_dim - position - 1] = np.round(layer_2)
        
        # store hidden layer so we can use it in the next timestep
        ##don't miss this
        layer_1_values.append(copy.deepcopy(layer_1))
    
    future_layer_1_delta = np.zeros(hidden_dim)
    
    for position in range(binary_dim):
        
        X = np.array([[a[position],b[position]]])
        layer_1 = layer_1_values[-position-1]
        prev_layer_1 = layer_1_values[-position-2]
        
        # error at output layer
        layer_2_delta = layer_2_deltas[-position-1]
        # error at hidden layer
        layer_1_delta = (future_layer_1_delta.dot(synapse_h.T) + layer_2_delta.dot(synapse_1.T)) * sigmoid_output_to_derivative(layer_1)

        # let's update all our weights so we can try again
        synapse_1_update += np.atleast_2d(layer_1).T.dot(layer_2_delta)
        synapse_h_update += np.atleast_2d(prev_layer_1).T.dot(layer_1_delta)
        synapse_0_update += X.T.dot(layer_1_delta)
        
        future_layer_1_delta = layer_1_delta
    

    synapse_0 += synapse_0_update * alpha
    synapse_1 += synapse_1_update * alpha
    synapse_h += synapse_h_update * alpha    

    synapse_0_update *= 0
    synapse_1_update *= 0
    synapse_h_update *= 0
    
    # print out progress
    if(j % 1000 == 0):
        print "Error:" + str(overallError)
        print "Pred:" + str(d)
        print "True:" + str(c)
        out = 0
        for index,x in enumerate(reversed(d)):
            out += x*pow(2,index)
        print str(a_int) + " + " + str(b_int) + " = " + str(out)
        print "------------"

Error:[[ 4.43556853]]
Pred:[0 0 0 0 0 0 0 0]
True:[0 1 1 0 1 1 1 0]
84 + 26 = 0
------------
Error:[[ 4.04943453]]
Pred:[1 0 1 1 0 1 0 1]
True:[0 1 0 1 1 1 1 0]
84 + 10 = 181
------------
Error:[[ 3.81363021]]
Pred:[1 1 1 1 1 1 1 1]
True:[0 1 1 0 1 0 1 1]
60 + 47 = 255
------------
Error:[[ 3.99675767]]
Pred:[1 0 0 0 0 1 1 0]
True:[1 1 0 0 0 0 0 0]
99 + 93 = 134
------------
Error:[[ 3.1408269]]
Pred:[0 1 1 0 1 1 1 1]
True:[0 1 1 0 1 0 1 1]
59 + 48 = 111
------------
Error:[[ 2.51276591]]
Pred:[1 0 0 0 1 0 1 1]
True:[1 0 0 0 1 0 1 1]
36 + 103 = 139
------------
Error:[[ 1.1066054]]
Pred:[0 1 0 0 1 0 1 0]
True:[0 1 0 0 1 0 1 0]
28 + 46 = 74
------------
Error:[[ 1.12224361]]
Pred:[0 1 0 0 0 0 0 1]
True:[0 1 0 0 0 0 0 1]
15 + 50 = 65
------------
Error:[[ 0.7802429]]
Pred:[1 0 0 0 0 0 1 0]
True:[1 0 0 0 0 0 1 0]
47 + 83 = 130
------------
Error:[[ 0.33935628]]
Pred:[0 1 1 0 1 1 0 0]
True:[0 1 1 0 1 1 0 0]
37 + 71 = 108
------------
